# Map_profiles.ipynb

In [1]:
import pandas as pd
import numpy as np 
import os
import matplotlib.pyplot as plt

In [4]:
total_df = pd.read_csv('../data/total_df.csv')

In [5]:
output_dir = "../data/profiles"
os.makedirs(output_dir, exist_ok=True)

grouped_meas = total_df.groupby('File Name')

# We'll collect all data into a list of dicts
image_data = []

for date, group in grouped_meas:
    group = group.sort_values(by='Depth (m)') 

    # Determine test type based on number of depth values
    test_type = "point" if group['Depth (m)'].nunique() <= 1 else "cast"

    if test_type == "cast":
        fig, axs = plt.subplots(1, 4, figsize=(20, 6), sharey=True)
        fig.suptitle(f'Profiles on {date}', fontsize=36)

        axs[0].plot(group['Temperature (°C)'], group['Depth (m)'])
        axs[0].set_xlabel('Temperature (°C)', fontsize=24, fontweight='bold')
        axs[0].set_ylabel('Depth (m)', fontsize=24, fontweight='bold')
        axs[0].invert_yaxis()

        axs[1].plot(group['Conductivity (µS/cm)'], group['Depth (m)'])
        axs[1].set_xlabel('Conductivity (µS/cm)', fontsize=24, fontweight='bold')

        axs[2].plot(group['Salinity (PSS)'], group['Depth (m)'])
        axs[2].set_xlabel('Salinity (PSS)', fontsize=24, fontweight='bold')

        axs[3].plot(group['Density (kg/m³)'], group['Depth (m)'])
        axs[3].set_xlabel('Density (kg/m³)', fontsize=24, fontweight='bold')

        for ax in axs:
            ax.tick_params(axis='both', labelsize=16)

        plt.tight_layout()

        img_path = os.path.join(output_dir, f"profile_{date}.png")
        fig.savefig(img_path)
        plt.close(fig)
    else:
        img_path = None  # No image for point measurement

    # Store info
    image_data.append({
        'File Name': date,
        'Latitude': group.iloc[0]['Latitude'],
        'Longitude': group.iloc[0]['Longitude'],
        'ImagePath': img_path,
        'TestType': test_type,
        'Temperature (°C)': group['Temperature (°C)'].iloc[0],
        'Conductivity (µS/cm)': group['Conductivity (µS/cm)'].iloc[0],
        'Salinity (PSS)': group['Salinity (PSS)'].iloc[0],
        'Density (kg/m³)': group['Density (kg/m³)'].iloc[0],
    })

# Create DataFrame
image_df = pd.DataFrame(image_data)
image_df.head()

,File Name,Latitude,Longitude,ImagePath,TestType,Temperature (°C),Conductivity (µS/cm),Salinity (PSS),Density (kg/m³)
0,CC2435009_20250513_181211,40.274148,-106.839900,../data/profiles\profile_CC2435009_20250513_18...,cast,9.153524,296.339874,0.204679,999.934361
1,CC2435009_20250513_181617,40.276507,-106.841752,../data/profiles\profile_CC2435009_20250513_18...,cast,9.902309,302.497925,0.204992,999.873001
2,CC2435009_20250513_182530,40.279905,-106.851733,../data/profiles\profile_CC2435009_20250513_18...,cast,12.425925,324.117126,0.206352,999.611949
3,CC2435009_20250513_182851,40.279532,-106.852019,../data/profiles\profile_CC2435009_20250513_18...,cast,12.494535,324.762451,0.206421,999.603737
4,CC2435009_20250513_194404,40.283628,-106.853450,../data/profiles\profile_CC2435009_20250513_19...,cast,12.716255,327.775024,0.207239,999.577265


In [6]:
image_df.to_csv('../data/profile_data.csv')